In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
# Define image parameters
IMG_HEIGHT = 1024
IMG_WIDTH = 1024
BATCH_SIZE = 32
NUM_CLASSES = 15

In [21]:
# Define the model
model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    Conv2D(16, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),

    # Conv2D(64, (3, 3), activation='relu'), -----> to reduce a bit the complexity of the initial model I commented these layers
    # MaxPooling2D((2, 2)),
    # BatchNormalization(),

    # Conv2D(128, (3, 3), activation='relu'),
    # MaxPooling2D((2, 2)),
    # BatchNormalization(),

    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [23]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_28 (Conv2D)          (None, 1022, 1022, 16)    160       
                                                                 
 max_pooling2d_28 (MaxPooli  (None, 511, 511, 16)      0         
 ng2D)                                                           
                                                                 
 batch_normalization_28 (Ba  (None, 511, 511, 16)      64        
 tchNormalization)                                               
                                                                 
 conv2d_29 (Conv2D)          (None, 509, 509, 16)      2320      
                                                                 
 max_pooling2d_29 (MaxPooli  (None, 254, 254, 16)      0         
 ng2D)                                                           
                                                      

In [24]:
# Train the model
history = model.fit(
    X_train,
    validation_data=X_val,
    epochs=5,
    callbacks=[early_stopping])

NameError: name 'X_train' is not defined

In [25]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')

NameError: name 'X_test' is not defined